In [304]:
import requests

# Auth headers
# https://docs.github.com/en/free-pro-team@latest/developers/apps/authorizing-oauth-apps#non-web-application-flow
# https://github.com/settings/tokens/new (repo read, commits aka not full priv)
headers = {'Authorization': 'token ZZZZZ', 'Accept': 'application/vnd.github.v3+json' }

baseurl = 'https://api.github.com/repos/MariaDB/server/'

query = baseurl + 'pulls'
# https://docs.github.com/en/free-pro-team@latest/rest/reference/pulls#list-pull-requests
# 100 per page is the maximum
response = requests.get(query, params={'accept': 'application/vnd.github.v3+json', 'state': 'open', 'sort': 'updated', 'per_page': 100}, headers=headers)

prs = response.json()

In [308]:
ghuser = 'grooverdan'
jirauser = 'danblack'

# pypy: jira
from jira import JIRA, JIRAError

jira = JIRA('https://jira.mariadb.org')
# https://jira.readthedocs.io/en/master/api.html#issue

import pprint, re
pp = pprint.PrettyPrinter(indent=4)

unassigned = []
reviewrequest = []
assigned = []
waiting_on_contributor = []
drafts = []

def prvalue(pr):
    if pr['title'].startswith('MDEV'):
        mdev = re.split('[:\s]', pr['title'], 2)[0]
        try:
            issue = jira.issue(mdev)
        except JIRAError as j:
            return (pr['number'], pr['html_url'], pr['title'], pr['updated_at'], j.text)
        issuef = issue.fields
        if issuef.assignee:
            iassignee =  issuef.assignee.displayName
        else:
            iassignee = None
        return (pr['number'], mdev, pr['html_url'], issue.permalink(), pr['title'], pr['updated_at'], iassignee)
    else:
        return (pr['number'], pr['html_url'], pr['title'], pr['updated_at'])


for pr in list(prs):
    if pr['draft'] or 'WIP' in pr['title']:
        drafts.append(prvalue(pr))
    elif [a for a in pr['assignees'] if a['login'] == ghuser]:
        if [lbl for lbl in pr['labels'] if lbl['name'] == 'Waiting_on_contributor_review_response']:
            waiting_on_contributor.append(prvalue(pr))
        else:
            assigned.append(prvalue(pr))
    elif [r for r in pr['requested_reviewers'] if r['login'] == ghuser]:
        reviewrequest.append(prvalue(pr))
    else:
        unassigned.append(prvalue(pr))
        
c = len(drafts) + len(waiting_on_contributor) + len(reviewrequest) + len(unassigned) + len(assigned)
    
print("prs {} , allocated {}".format(len(prs), c))

prs 90 , allocated 87


In [309]:
reviewrequest


[]

In [310]:
assigned

[(1643,
  'MDEV-23299',
  'https://github.com/MariaDB/server/pull/1643',
  'https://jira.mariadb.org/browse/MDEV-23299',
  'MDEV-23299: The udf_init() function cause server crash.',
  '2020-09-13T08:56:56Z',
  'Daniel Black'),
 (1651,
  'https://github.com/MariaDB/server/pull/1651',
  ' MDEV-23485: Change table to merge engine may cause table data lost.',
  '2020-10-05T00:48:32Z'),
 (1556,
  'MDEV-22659',
  'https://github.com/MariaDB/server/pull/1556',
  'https://jira.mariadb.org/browse/MDEV-22659',
  'MDEV-22659: Create one single unified and optimal logrotate config (pending on RPM packaging changes)',
  '2020-11-13T12:21:17Z',
  'Daniel Black')]

In [311]:
unassigned

[(616,
  'MDEV-8379',
  'https://github.com/MariaDB/server/pull/616',
  'https://jira.mariadb.org/browse/MDEV-8379',
  'MDEV-8379: example config file for multi-instance (from openSUSE)',
  '2019-03-08T09:36:07Z',
  'Robert Bindar'),
 (604,
  'MDEV-9158',
  'https://github.com/MariaDB/server/pull/604',
  'https://jira.mariadb.org/browse/MDEV-9158',
  'MDEV-9158: file_key_management should support reading from a named pipe',
  '2019-03-08T09:36:40Z',
  'Sergei Golubchik'),
 (592,
  'MDEV-11045',
  'https://github.com/MariaDB/server/pull/592',
  'https://jira.mariadb.org/browse/MDEV-11045',
  'MDEV-11045 systemd reload / rpm {pre|post}{inst|uninst}',
  '2019-03-08T09:37:40Z',
  'Sergei Golubchik'),
 (1364,
  'https://github.com/MariaDB/server/pull/1364',
  'support row based replication, read and write of MySQL 5.7.x JSON binary',
  '2019-08-15T05:36:12Z'),
 (631,
  'MDEV-15547',
  'https://github.com/MariaDB/server/pull/631',
  'https://jira.mariadb.org/browse/MDEV-15547',
  'MDEV-15547

In [312]:
waiting_on_contributor

[]

In [313]:
drafts

[(1536,
  'https://github.com/MariaDB/server/pull/1536',
  'WIP: Implement service-test.sh to help verify sysvinit/systemd',
  '2020-05-14T08:59:03Z'),
 (1555,
  'MDEV-19168',
  'https://github.com/MariaDB/server/pull/1555',
  'https://jira.mariadb.org/browse/MDEV-19168',
  'MDEV-19168 Add ssl-flush command',
  '2020-05-27T14:56:36Z',
  'Robert Bindar'),
 (1501,
  'MDEV-22221',
  'https://github.com/MariaDB/server/pull/1501',
  'https://jira.mariadb.org/browse/MDEV-22221',
  'MDEV-22221: wolfssl tls13',
  '2020-05-29T02:08:33Z',
  'Vladislav Vaintroub'),
 (1634,
  'MDEV-19177',
  'https://github.com/MariaDB/server/pull/1634',
  'https://jira.mariadb.org/browse/MDEV-19177',
  'MDEV-19177: Geometry support by the partition feature',
  '2020-07-24T07:44:19Z',
  'Sergei Golubchik'),
 (1637,
  'https://github.com/MariaDB/server/pull/1637',
  '[WIP] MDEV-23143: Adding a JSON_EQUALS function',
  '2020-07-27T14:54:19Z'),
 (1506,
  'https://github.com/MariaDB/server/pull/1506',
  'WIP: Deb: Inc